In [20]:
import pyodbc
import pandas as pd
import time
from datetime import datetime

In [21]:
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=FREYJA\SQLEXPRESS;'
                      'Database=tradingCrypto;'
                      'Trusted_Connection=yes;')

cursor = conn.cursor()
cursor.execute('SELECT * FROM [dbo].[cryptoPrices_raw]')

for i in cursor:
    print(i)

('1641535343306', 'bitcoin', '41720.44267405448')
('1642050250941', 'bitcoin', '43692.499996266684')
('1642564980424', 'bitcoin', '41779.01270893919')
('1643079784619', 'bitcoin', '36376.901774691425')
('1643598097468', 'bitcoin', '37051.64434316899')
('1644109336146', 'bitcoin', '41599.71576444385')
('1644638467598', 'bitcoin', '42382.93296316678')
('1645156908928', 'bitcoin', '40791.33301969015')
('1645671614660', 'bitcoin', '36694.076166224695')
('1646182872798', 'bitcoin', '44204.669171661764')
('1646694112125', 'bitcoin', '38171.92522563868')
('1647205323692', 'bitcoin', '38717.46688416726')
('1647720147327', 'bitcoin', '42037.48112784869')
('1648234923153', 'bitcoin', '44445.73225084325')
('1648749739255', 'bitcoin', '45847.37365494933')
('1641481437612', 'ethereum', '3405.411317944101')
('1641985403280', 'ethereum', '3259.829386022108')
('1642489343351', 'ethereum', '3212.2708855398632')
('1642993205950', 'ethereum', '2447.1870210740426')
('1643500895566', 'ethereum', '2603.6038

C:\Users\katze\AppData\Local\Temp\ipykernel_35544\245314679.py:10: DeprecationWarning: PyUnicode_FromUnicode(NULL, size) is deprecated; use PyUnicode_New() instead
  print(i)


In [22]:
query = "SELECT * FROM [dbo].[cryptoPrices_raw];"
dfStg = pd.read_sql(query, conn)

print(dfStg.head(5))

      timestampp  id_coin            priceUSD
0  1641535343306  bitcoin   41720.44267405448
1  1642050250941  bitcoin  43692.499996266684
2  1642564980424  bitcoin   41779.01270893919
3  1643079784619  bitcoin  36376.901774691425
4  1643598097468  bitcoin   37051.64434316899


C:\Users\katze\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [23]:
# Add a column for the primary key
dfStg = dfStg.reset_index()
dfStg = dfStg.rename(columns={'index':'id'})  
dfStg['id'] = dfStg.index + 1
print(dfStg)

          id     timestampp  id_coin            priceUSD
0          1  1641535343306  bitcoin   41720.44267405448
1          2  1642050250941  bitcoin  43692.499996266684
2          3  1642564980424  bitcoin   41779.01270893919
3          4  1643079784619  bitcoin  36376.901774691425
4          5  1643598097468  bitcoin   37051.64434316899
...      ...            ...      ...                 ...
10792  10793  1648774916773     aave  216.93135126732813
10793  10794  1648778444415     aave  210.76022407196155
10794  10795  1648782022535     aave  209.19469069851968
10795  10796  1648785642123     aave  216.03915646202725
10796  10797  1648789297353     aave  219.78452154994082

[10797 rows x 4 columns]


In [24]:
print(dfStg.dtypes)

id             int64
timestampp    object
id_coin       object
priceUSD      object
dtype: object


In [25]:
# Transforming column timestampp and priceUSD
#dfStg["priceUSD"] = pd.to_numeric(df["a"])
dfStg = dfStg.astype({'priceUSD':'float'})
dfStg['timestampp'] = dfStg['timestampp'].str[:10]
dfStg['timestampp'] = pd.to_datetime(dfStg['timestampp'],unit='s')
dfStg.sort_values(by='timestampp') 
print(dfStg)
 

          id          timestampp  id_coin      priceUSD
0          1 2022-01-07 06:02:23  bitcoin  41720.442674
1          2 2022-01-13 05:04:10  bitcoin  43692.499996
2          3 2022-01-19 04:03:00  bitcoin  41779.012709
3          4 2022-01-25 03:03:04  bitcoin  36376.901775
4          5 2022-01-31 03:01:37  bitcoin  37051.644343
...      ...                 ...      ...           ...
10792  10793 2022-04-01 01:01:56     aave    216.931351
10793  10794 2022-04-01 02:00:44     aave    210.760224
10794  10795 2022-04-01 03:00:22     aave    209.194691
10795  10796 2022-04-01 04:00:42     aave    216.039156
10796  10797 2022-04-01 05:01:37     aave    219.784522

[10797 rows x 4 columns]


In [26]:
print(dfStg.dtypes)

id                     int64
timestampp    datetime64[ns]
id_coin               object
priceUSD             float64
dtype: object


In [28]:
# Insert Dataframe into SQL Server:
for index, row in dfStg.iterrows():
     cursor.execute("INSERT INTO dbo.cryptoPrices_st (id,timestampp,id_coin,priceUSD) values(?,?,?,?)", row.id, row.timestampp, row.id_coin, row.priceUSD)
conn.commit()
cursor.close()
# see total time to do insert
#print("%s seconds ---" % (time.time() - start_time))